In [1]:
import os
import sys
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
#import xlsxwriter
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [5]:
import pandas_gbq

In [41]:
import numpy_financial as npf

## Ferratum AU Oct 2021

In [23]:
###Manual: adjust book_id and region
book_id='3435712288'
region='au' ###'au' or 'eu'

In [13]:
query_purchase='''SELECT book_id,actual_purchase_price*-1 as purchase_local_ccy,settlement_date
                FROM presentation.dim_books db
                where 1=1
                and book_id='{}' '''.format(book_id)

In [15]:
df_purch = pandas_gbq.read_gbq(query_purchase,project_id="data-team-318806")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [171]:
df_purch

,book_id,purchase_local_ccy,settlement_date
0,3435712288,-659683.0,2021-10-20


In [206]:
if region=='au':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_au
            where 1=1
            and book_id='{}' '' '''.format(book_id)
elif region=='eu':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_eu
            where 1=1
            and book_id='{}' '' '''.format(book_id)
else:
    query_cf=None


In [208]:
df_cf=pandas_gbq.read_gbq(query_cf,project_id="data-team-318806")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [209]:
df_cf

,book_id,book_name,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,...,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01
0,3435712288,Ferratum / Multitude Oct 21,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,2813.17,3277.00,3851.00,2848.79,2514.33,1219.49,973.61,1655.67,1200.00,1156.45
1,3435712288,Ferratum / Multitude Oct 21,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,16053.32,4703.93,7357.25,5227.49,1559.51,1789.87,634.38,896.98,-421.64,935.07
2,3435712288,Ferratum / Multitude Oct 21,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,179.00,80.00,90.00,130.00,68.00,40.00,20.00,20.00,20.00,20.00
3,3435712288,Ferratum / Multitude Oct 21,261402.0,49947.22,120861.03,155443.49,193742.23,121042.02,115673.38,92877.99,...,3814.20,3855.34,3015.16,2890.92,2541.77,3612.70,3260.36,4032.07,3602.80,4215.02


In [210]:
df_cf_agg = df_cf.groupby(['book_id', 'book_name']).sum().reset_index().copy()

In [211]:
df_cf_agg

,book_id,book_name,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,...,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01
0,3435712288,Ferratum / Multitude Oct 21,261402.0,49947.22,120861.03,155443.49,193742.23,121042.02,115673.38,92877.99,...,22859.69,11916.27,14313.41,11097.2,6683.61,6662.06,4888.35,6604.72,4401.16,6326.54


In [216]:
book_name=df_cf_agg.iloc[0,1]

In [218]:
cf_values = df_cf_agg.iloc[0, 2:].tolist()

In [220]:
relevant_months=df_cf_agg.columns[2:].tolist()

In [222]:
df_agg_cf_format=pd.DataFrame({'collection':cf_values,'month':relevant_months})

In [224]:
df_agg_cf_format['month_cohort']=range(1, len(df_agg_cf_format)+1)


In [226]:
df_agg_cf_format.head(2)

,collection,month,month_cohort
0,261402.00,2021-09-01,1
1,49947.22,2021-10-01,2


In [228]:
cashflow=[df_purch.iloc[0,1]]+df_agg_cf_format['collection'].tolist()

In [232]:
cashflow=[round(value, 2) for value in cashflow]

In [ ]:
#########################################

In [242]:
df_agg_cf_format.to_csv('verification.csv', index=False)

In [238]:
npf.irr(cashflow) * 100

19.174297894368507

In [240]:
df_agg_cf_format

,collection,month,month_cohort
0,261402.00,2021-09-01,1
1,49947.22,2021-10-01,2
2,120861.03,2021-11-01,3
3,155443.49,2021-12-01,4
4,193742.23,2022-01-01,5
5,121042.02,2022-02-01,6
6,115673.38,2022-03-01,7
7,92877.99,2022-04-01,8
8,98480.75,2022-05-01,9
9,75647.75,2022-06-01,10
